# Ejercicio de machine learning 1: detección de conexiones cibernéticas intrusivas

En este ejercicio (mucho menos guiado que los anteriores), tu objetivo es construir un detector de intrusiones cibernéticas.

Para ello, utilizarás un conocido (pero modificado) dataset conocido como el Dataset [KDD-CUP-99](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html). Deberás construir un detector de intrusiones de red: un modelo de machine learning que sea capaz de distinguir entre conexiones "maliciosas" (`attacks`) y conexiones "buenas" (`normal`es, corrientes y legítimas).

## El dataset

El dataset contiene casi 20.000 conexiones (una por fila/observación) recopiladas a lo largo de un período de tiempo, y auditadas por expertos en ciberseguridad; de forma que los expertos han etiquetado cada conexión como `attack` o `normal` en base a si es una conexión maliciosa o una normal. Dado que se trata de una tarea muy rutinaria y aburrida, sería ideal utilizar los datos ya etiquetados por expertos para ser capaz de entrenar un modelo de machine learning que pueda predecir en conexiones futuras si se tratan de ataques o no, de forma automática.

Cada fila del CSV (`KDD_dataset.csv`, separado por punto y coma) contiene por tanto información de una conexión. Aparte de aparecer si se trata o no de un ataque (que es la variable a predecir `type`, la última columna), cada conexión contiene muchas columnas con información adicional de la conexión. Puedes (¡y debes!) usar estas columnas, con el objetivo de predecir lo mejor posible si dicha conexión se trata de un ataque o no en base a dichas features.

El dataset contiene por tanto 46 columnas: 45 atributos de la conexión, y la variable a predecir `type`. La descripción de lo que significa cada columna puedes verla en la siguiente tabla (la cual he construido a partir de la [información oficial del dataset](http://kdd.ics.uci.edu/databases/kddcup99/task.html)):

| columna (o *familia* de columnas)  | descripción  |
|---|---|
|duration | length (number of seconds) of the connection |
|protocol_type| type of the protocol, e.g. tcp, udp, etc. |
| service | network service on the destination, e.g., http, telnet, etc. |
| src_bytes | number of data bytes from source to destination |
|dst_bytes| number of data bytes from destination to source |
|flag | normal or error status of the connection |
| land | 1 if connection is from/to the same host/port; 0 otherwise | 
| wrong_fragment | number of "wrong" fragments |
| urgent | number of urgent packets |
|hot | number of "hot" indicators|
|num_failed_logins | number of failed login attempts |
| logged_in | 1 if successfully logged in; 0 otherwise |
| num_compromised | number of "compromised" conditions |
|root_shell | 1 if root shell is obtained; 0 otherwise |
|su_attempted| 1 if "su root" command attempted; 0 otherwise |
|num_root |number of "root" accesses  |
|num_file_creations |number of file creation operations  |
|num_shells  | number of shell prompts |
|num_access_files | number of operations on access control files |
|num_outbound_cmds | number of outbound commands in an ftp session |
|is_hot_login | 1 if the login belongs to the "hot" list; 0 otherwise |
|is_guest_login | 1 if the login is a "guest" login; 0 otherwise |
|count | number of connections to the same host as the current connection in the past two seconds |
|srv_count | number of connections to the same service as the current connection in the past two seconds |
|serror_rate | % of same-host connections that have "SYN" errors |
|srv_serror_rate |% of same-service connections that have "REJ" errors  |
| rerror_rate| % of same-host connections that have "REJ" errors |
| srv_rerror_rate| % of same-service connections that have "REJ" errors |
|same_srv_rate |% of connections to the same service  |
|diff_srv_rate |% of connections to different services  |
| srv_diff_host_rate| % of connections to different hosts |

Si miras la tabla de arriba, verás que no tiene 45 entradas, sino que son algunas menos. Esto se debe a que, a partir de alguna de las entradas, hemos generado por ti más de una feature que pueden usar tus modelos. Por ejemplo: en la tabla de arriba puedes ver que hay una entrada que es `protocol_type`; que es, según la descripción, el tipo de protocolo de red que ha usado la conexión: tcp, udp, etcétera. Nosotros hemos hecho *One-Hot-Encoding* de este tipo de variables por ti; de modo que en vez de encontrar una columna que es `protocol_type`, te vas a encontrar `protocol_type__tcp`, `protocol_type__udp` y `protocol_type__icmp`; y cada una de ellas tiene valor de 1 (si se trata efectivamente de una conexión de ese tipo), y de 0 si no e así.

Por ejemplo: para las primeras dos conexiones del dataset nos encontramos:

|protocol_type__tcp |	protocol_type__udp |	protocol_type__icmp|
|---|---|---|
|0 |1|0
| 1 | 0|0

Lo cual quiere decir que la primera conexión se trata de una udp; y la segunda de una tcp. De esta forma, lo que originariamente sería solo una feature de tipo de dato string (`protocol_type`, que tomaría valores de strings como `tcp`, `udp` o `icmp`), ahora son varias features; pero numéricas, que podemos meter directamente en los modelos de machine learning. Puedes leer más sobre One-Hot-Encoding en la versión extendida de las diapositivas de machine learning.

Sobra decir que realmente no tienes por qué saber nada de ciberseguridad para poder hacerlo, ni tienes que entender qué es exactamente cada feature; asumiendo que las features están bien recopiladas, una de las *gracias* del machine learning es que puedes ser capaz de realizar acciones basadas en datos sin tener conocimiento extenso del campo donde lo aplicas.

## Objetivos

1. Pensar en **qué ténica de machine learning** debes utilizar. ¿Se trata de aprendizaje supervisado, o no supervisado? ¿Y dentro de una de esas dos?
2. Dependiendo de lo que respondas a la pregunta anterior, es posible que debas seguir alguno de los workflows que vimos en la teoría de machine learning en clase... Análisis exploratorio de los datos, separar en train y test, separar también validación (o usar cross-validation), probar distintos modelos e hiperparámetros... Lo que necesites para construir un modelo que sea capaz de **detectar lo mejor posible si una conexión se trata de una intrusión o no**.
3. Una vez construido el modelo, no esperes que el equipo de ciberseguridad te dé luz verde para empezar a usarlo mañana mismo. Tanto tú como tu modelo tenéis que **ganaros la confianza** de ser realmente útiles. Para ello, va a ser necesario que demuestres cómo de bien podrá, tu modelo, detectar nuevas intrusiones a futuro. O, dicho de otra forma, mostrar **cómo de bueno/malo esperamos que sea el modelo a futuro**.
4. Justo esta mañana hemos recibido tres registros de nuevas conexiones, que el departamento de ciberseguridad no tiene tiempo ahora mismo para comprobar. ¡Es la oportunidad ideal para comenzar a usar tu modelo! **Genera predicciones** para esas tres nuevas observaciones (que evidentemente no contienen la etiqueta `type`, ya que de momento desconocemos si son intrusiones o no), las cuales están disponibles en el archivo `nuevas_conexiones.csv` y guárdalas de vuelta en un archivo CSV que se llame `predicciones_nuevas_conexiones.csv` que incluya todas las features de dichas observaciones junto con una nueva columna llamada `prediccion_ml`, que contenga las predicciones.

<div class="alert alert-warning">Como es normal, la mayoría de conexiones son "buenas", es decir, no son ataques. Así podrás ver que ocurre en el dataset. Es posible que este fenómeno influya en la manera en la que quieres medir cómo comparar qué modelo de los que pruebes lo hace mejor...</div>

<div class="alert alert-warning">Para el paso 4, verás que el formato del CSV de las nuevas conexiones es ligeramente distinto al del dataset original. A la hora de hacer predicciones, es importante "pasar" al modelo las features en el mismo orden que con el que entrenó; si no, ¡confundirás al modelo porque pensará que una feature es realmente otra, y viceversa!</div>

Y nada más. ¡Buena suerte!